# Lesson 1: Router Engine

In [ ]:
%pip install python-dotenv llama-index llama-index-llms-ollama llama-index-embeddings-ollama

In [2]:
import nest_asyncio

nest_asyncio.apply()

## Load data

In [3]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

## Define LLM and Embedding model

In [5]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [8]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.llm = Ollama(model="llama3.2")
Settings.embed_model = OllamaEmbedding(model_name="nomic-embed-text")



## Define Summary Index and Vector Index over the Same Data

In [9]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

## Define Query Engines and Set Metadata

In [11]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)

vector_query_engine = vector_index.as_query_engine()

In [13]:
from llama_index.core.tools import QueryEngineTool

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to the MetaGPT paper"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper"
    ),
)

## Define Router Query Engine

In [14]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [15]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 1: This choice is most relevant because it allows for the retrieval of specific context from a document, which can be useful for summarization questions..
The document appears to be a technical documentation or guide for developing a drawing application. It outlines the required Python third-party packages, other language dependencies, and provides an overview of the full API specification. Additionally, it explains the logic analysis, which includes a list of files responsible for implementing various components of the application, such as the main application loop, tool selection interface, color selection interface, file saving and opening functionality.


In [19]:
response = query_engine.query(
    "How do agents share information with other agents?"
)
print(str(response))

Selecting query engine 1: The MetaGPT paper discusses multi-agent environments and how agents can share information to achieve common goals..
Agents use a shared message pool to publish structured messages. They can also subscribe to relevant messages based on their profiles.


In [20]:
response = query_engine.query(
    "Tell me about the ablation study results?"
)
print(str(response))

Selecting query engine 1: Can be used to retrieve specific context from the MetaGPT paper, allowing users to find information about ablation study results..
The collaboration between MetaGPT and GPT-4 results in improved performance on the HumanEval benchmark.
